In [196]:
import pandas as pd

# Import of Data
# Source: https://www.bmi.gv.at/412/Nationalratswahlen/Nationalratswahl_2019/
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
df = pd.read_csv("NRW19.csv", sep=';',thousands = '.', decimal = ',')
# Amtliche Gemeindekennzahl (GKZ)

In [197]:
# Data Preparation
df_reduced = df

## Remove all "Wahlkarten"
df_reduced = df_reduced[~df_reduced.GKZ.str.endswith('99')]
## Remove all aggregated results
df_reduced = df_reduced[~df_reduced.GKZ.str.endswith('00')]


## we will only need the percentages and "GKZ", "Gebietsname", "Stimmen"
import numpy
party_names = df_reduced.columns[numpy.append(df_reduced.columns.str.startswith('%')[1:], [False])]
percentages_names = df_reduced.columns[df_reduced.columns.str.startswith('%')]
df_reduced = df_reduced.drop(party_names, axis=1)
df_reduced.rename(columns=dict(zip(percentages_names, party_names)),inplace=True)
df_reduced = df_reduced.drop(['Wahlbe-rechtigte', 'Ungültige', 'Gültige'], axis=1)
# add a column with the respective name of the state

## Aggregate (i.e., sum up) all the percentages of all small parties
small_parties_name = ['KPÖ', 'WANDL', 'BZÖ', 'BIER', 'CPÖ', 'GILT', 'SLP']
df_reduced['others'] = df_reduced[small_parties_name].sum(axis=1)
df_reduced = df_reduced.drop(small_parties_name, axis=1)


#pd.set_option('display.max_rows', 8)
df_reduced

,GKZ,Gebietsname,Stimmen,ÖVP,SPÖ,FPÖ,NEOS,JETZT,GRÜNE,others
8,G10101,Eisenstadt,7192,41.69,20.38,14.38,7.52,1.69,13.27,1.07
11,G10201,Rust,1055,34.23,30.38,20.15,5.01,1.16,7.91,1.16
14,G10301,Breitenbrunn am Neusiedler See,1137,33.87,29.82,13.42,6.94,3.42,11.89,0.63
15,G10302,Donnerskirchen,1076,44.38,22.85,18.89,5.67,1.23,6.33,0.66
...,...,...,...,...,...,...,...,...,...,...
2436,G92001,Brigittenau,22535,20.07,36.48,14.62,5.83,2.92,17.83,2.25
2439,G92101,Floridsdorf,56568,25.99,30.49,20.13,6.65,2.84,11.94,1.95
2442,G92201,Donaustadt,69291,25.83,29.42,18.26,7.78,2.96,13.79,1.97
2445,G92301,Liesing,41240,28.19,27.35,15.93,9.67,2.69,14.50,1.68


In [199]:
# 

8       1
11      1
14      1
15      1
       ..
2436    9
2439    9
2442    9
2445    9
Name: GKZ, Length: 2118, dtype: object

In [214]:
def find_state (value_str):
    value = int(value_str)
   if value == 1:
      return 'Burgenland'
   if value == 2:
      return 'Kärnten'
   if value == 3:
      return 'Niederösterreich'
   if value == 4:
      return 'Oberösterreich'
   if value == 5:
      return 'Salzburg'
   if value == 6:
      return 'Steiermark.'
   if value == 7:
      return 'Tirol'
   if value == 8:
      return 'Vorarlberg'
   if value == 9:
      return 'Wien'
   return 'Other'

In [217]:
df_reduced['state'] = df_reduced.GKZ.str[1].apply(find_state)

df_reduced.GKZ.str[1]

TypeError: cannot convert the series to <class 'int'>

In [216]:
df_reduced['state']

8       Burgenland
11      Burgenland
14      Burgenland
15      Burgenland
           ...    
2436         Other
2439         Other
2442         Other
2445         Other
Name: state, Length: 2118, dtype: object

In [213]:
x = df_reduced.GKZ.str[1]
x

8       1
11      1
14      1
15      1
       ..
2436    9
2439    9
2442    9
2445    9
Name: GKZ, Length: 2118, dtype: object